## Attention 신경망 구현 및 학습

## konlpy 설치 

<Windows 인 경우>
- 참고 : http://konlpy.org/ko/latest/install/ 

<Mac 인 경우>
1. JVM 설치 (참조 : https://blog.naver.com/PostView.nhn?blogId=coldoutside&logNo=221449433535&from=search&redirect=Log&widgetTypeCall=true&directAccess=false )
2. xcode-select --install
3. export MACOSX_DEPLOYMENT_TARGET=10.10
4. CFLAGS='-stdlib=libc++' pip install konlpy

In [1]:
import random
import tensorflow as tf
from konlpy.tag import Okt

## 하이퍼 파라미터

In [2]:
EPOCHS = 200
NUM_WORDS = 2000

## Encoder

In [3]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_state=True)

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        _, h, c = self.lstm(x)
        return h, c

## Decoder

In [4]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        x, h, c = inputs
        x = self.emb(x)
        x, h, c = self.lstm(x, initial_state=[h, c])
        return self.dense(x), h, c

## Seq2seq

In [5]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos

    def call(self, inputs, training=False, mask=None):
        if training is True:
            x, y = inputs
            h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c))
            return y
        else:
            x = inputs
            h, c = self.enc(x)
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64)

            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c])
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [6]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비


In [7]:
dataset_file = 'chatbot_data.csv'  # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

/home/jeyong/.conda/envs/tf2/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [8]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [9]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

W0922 09:42:08.512909 140618365970240 deprecation.py:323] From /home/jeyong/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1220: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1, Loss: 3.0273499488830566, Accuracy: 89.98668670654297
Epoch 2, Loss: 0.6014062166213989, Accuracy: 90.96961212158203
Epoch 3, Loss: 0.5653008222579956, Accuracy: 91.04401397705078
Epoch 4, Loss: 0.5494084358215332, Accuracy: 91.11841583251953
Epoch 5, Loss: 0.5412740111351013, Accuracy: 91.10667419433594
Epoch 6, Loss: 0.5360599160194397, Accuracy: 91.08708953857422
Epoch 7, Loss: 0.5315815210342407, Accuracy: 91.12625122070312
Epoch 8, Loss: 0.5269421339035034, Accuracy: 91.1614990234375
Epoch 9, Loss: 0.5218955278396606, Accuracy: 91.20065307617188
Epoch 10, Loss: 0.5152804851531982, Accuracy: 91.2202377319336
Epoch 11, Loss: 0.5063177347183228, Accuracy: 91.23981475830078
Epoch 12, Loss: 0.49390506744384766, Accuracy: 91.28681182861328
Epoch 13, Loss: 0.47931432723999023, Accuracy: 91.44737243652344
Epoch 14, Loss: 0.46515291929244995, Accuracy: 91.72541046142578
Epoch 15, Loss: 0.4509647786617279, Accuracy: 92.089599609375
Epoch 16, Loss: 0.44248345494270325, Accuracy: 92.

Epoch 129, Loss: 0.08461929112672806, Accuracy: 98.34351348876953
Epoch 130, Loss: 0.0824970081448555, Accuracy: 98.36701202392578
Epoch 131, Loss: 0.0811024159193039, Accuracy: 98.38658905029297
Epoch 132, Loss: 0.07963287085294724, Accuracy: 98.44141387939453
Epoch 133, Loss: 0.07849061489105225, Accuracy: 98.46491241455078
Epoch 134, Loss: 0.0771343857049942, Accuracy: 98.48448944091797
Epoch 135, Loss: 0.07584907114505768, Accuracy: 98.48841094970703
Epoch 136, Loss: 0.07422827184200287, Accuracy: 98.4962387084961
Epoch 137, Loss: 0.07262223958969116, Accuracy: 98.50798797607422
Epoch 138, Loss: 0.07090356200933456, Accuracy: 98.56672668457031
Epoch 139, Loss: 0.06922190636396408, Accuracy: 98.60197448730469
Epoch 140, Loss: 0.06757397949695587, Accuracy: 98.63330078125
Epoch 141, Loss: 0.06619498133659363, Accuracy: 98.6724624633789
Epoch 142, Loss: 0.06506027281284332, Accuracy: 98.68421173095703
Epoch 143, Loss: 0.06382487714290619, Accuracy: 98.71553802490234
Epoch 144, Loss: 0

## 테스트 루프

In [10]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text)
    print('a: ', gt_text)
    print('p: ', texts)

_
q:  ['여기 기프티콘 되죠 \n']
a:  ['\t 네 현금영수증 해드릴까 요 \n']
p:  ['여기 진동 벨 로 알려 드리겠습니다 \n']
_
q:  ['네 에 테이크 아웃 도 가능한가요 \n']
a:  ['\t 네 로 오시 면 테이크 아웃 잔 에 담아 드려요 \n']
p:  ['아뇨 현재 법적 으로 금지 하고 있어요 \n']
_
q:  ['아메리카노 톨 사이즈 로 주세요 \n']
a:  ['\t 따뜻한 거 로 드릴 까요 \n']
p:  ['둘 다 아이스 인가요 \n']
_
q:  ['진동 을 따로 주시나요 \n']
a:  ['\t 주 번호 로 드리겠습니다 \n']
p:  ['네 기다리시면 안내 해드릴게요 \n']
_
q:  ['자리 있나요 \n']
a:  ['\t 네 있습니다 \n']
p:  ['네 배달 비 3000원 입니다 \n']
_
q:  ['그럼 루이보스 밀크 티 하나 \n']
a:  ['\t 네 알겠습니다 \n']
p:  ['네 디카 페인 라테 아메리카노 주문 가능합니다 \n']
_
q:  ['다음 에 무료 로 하고 엔 도장 찍어주세요 \n']
a:  ['\t 네 \n']
p:  ['네 카페라떼 컵 사이즈 는 뭘 로 드릴 까요 \n']
_
q:  ['아메리카노 한 잔 에 얼마 죠 \n']
a:  ['\t 입니다 \n']
p:  ['4000원 입니다 \n']
_
q:  ['얼마나 \n']
a:  ['\t 바로 만들어 드릴게요 \n']
p:  ['네 고객 님 결제 완료 되었습니다 \n']
_
q:  ['카푸치노 는 로 주시 고 아메리카노 는 로 \n']
a:  ['\t 네 더 없으세요 \n']
p:  ['드시고 가시나요 \n']
_
q:  ['아메리카노 는 어떤 종류 가 있나요 \n']
a:  ['\t 디카 페인 과 기본 아메리카노 2 종류 있습니다 \n']
p:  ['피지 와 나 티 종류 가 있습니다 \n']
_
q:  ['카카오 페이 로 결제 가능한가요 \n']
a:  ['\t 네 가능합니다 \n']
p:  ['네 2시 까지 가능하십니다 

_
q:  ['포인트 적립 할게요 \n']
a:  ['\t 네 결제 되셨습니다 \n']
p:  ['네 카드 도 와 드리겠습니다 \n']
_
q:  ['티라미수 는 있나요 \n']
a:  ['\t 네 티라미수 는 있습니다 \n']
p:  ['네 고객 님 티 종류 다 아이스 가능합니다 \n']
_
q:  ['네 현금영수증 해주세요 \n']
a:  ['\t 네 드시고 가시나요 \n']
p:  ['네 번호 입력 해주세요 \n']
_
q:  ['샷 추가 해주세요 \n']
a:  ['\t 네 알겠습니다 \n']
p:  ['네 디카 페인 라테 아메리카노 주문 가능합니다 \n']
_
q:  ['얼마 에요 \n']
a:  ['\t 만 원 입니다 \n']
p:  ['베이글 과 동일하게 2000원 입니다 \n']
_
q:  ['아이스 아메리카노 랑 샌드위치 주세요 \n']
a:  ['\t 10시 에 세트 할인 가능하세요 \n']
p:  ['아이스 아메리카노 4000원 입니다 \n']
